# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

/opt/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get data from `.csv` files and then preprocess data

In [2]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_en = pd.read_csv("data/movie_reviews_eng.csv")
    df_en.columns = ['Title', 'Year', 'Synopsis', 'Review']
    df_en.insert(0, 'Original Language', 'En')
        
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_fr.columns = ['Title', 'Year', 'Synopsis', 'Review']
    df_fr.insert(0, 'Original Language', 'Fr')
        
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    df_sp.columns = ['Title', 'Year', 'Synopsis', 'Review']
    df_sp.insert(0, 'Original Language', 'Sp')
        
    return pd.concat([df_en, df_fr, df_sp])

df = preprocess_data()

In [3]:
df.sample(10)

,Original Language,Title,Year,Synopsis,Review
1,En,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su..."
6,Fr,La Tour Montparnasse Infernale,2001,Deux employés de bureau incompétents se retrou...,"""Je ne peux pas croire que j'ai perdu du temps..."
4,Fr,Le Fabuleux Destin d'Amélie Poulain,2001,Cette comédie romantique raconte l'histoire d'...,"""Le Fabuleux Destin d'Amélie Poulain est un fi..."
9,Fr,Babylon A.D.,2008,"Dans un futur lointain, un mercenaire doit esc...","""Ce film est un gâchis complet. Les personnage..."
8,En,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of..."
7,Fr,Astérix aux Jeux Olympiques,2008,Dans cette adaptation cinématographique de la ...,"""Ce film est une déception totale. Les blagues..."
3,Fr,Les Choristes,2004,Ce film raconte l'histoire d'un professeur de ...,"""Les Choristes est un film magnifique qui vous..."
7,Sp,El Bar,2017,Un grupo de personas quedan atrapadas en un ba...,"""El Bar es una película ridícula y sin sentido..."
0,Sp,Roma,2018,Cleo (Yalitza Aparicio) es una joven empleada ...,"""Roma es una película hermosa y conmovedora qu..."
6,En,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi..."


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [4]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = 'Helsinki-NLP/opus-mt-fr-en'
es_en_model_name = 'Helsinki-NLP/opus-mt-es-en'
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt")

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

/opt/venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [5]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews = df.loc[df['Original Language'] == 'Fr']['Review'].to_list()
fr_reviews_en = [translate(text=r, model=fr_en_model, tokenizer=fr_en_tokenizer) for r in fr_reviews]

# filter synopsis in French and translate to English
fr_synopsis = df.loc[df['Original Language'] == 'Fr']['Synopsis'].to_list()
fr_synopsis_en = [translate(text=r, model=fr_en_model, tokenizer=fr_en_tokenizer) for r in fr_synopsis]

# filter reviews in Spanish and translate to English
es_reviews = df.loc[df['Original Language'] == 'Sp']['Review'].to_list()
es_reviews_en = [translate(text=r, model=es_en_model, tokenizer=es_en_tokenizer) for r in es_reviews]

# filter synopsis in Spanish and translate to English
es_synopsis = df.loc[df['Original Language'] == 'Sp']['Synopsis'].to_list()
es_synopsis_en = [translate(text=r, model=es_en_model, tokenizer=es_en_tokenizer) for r in es_synopsis]

# update dataframe with translated text
df.loc[df['Original Language'] == 'Fr', 'Review'] = fr_reviews_en
df.loc[df['Original Language'] == 'Fr', 'Synopsis'] = fr_synopsis_en
df.loc[df['Original Language'] == 'Sp', 'Review'] = es_reviews_en
df.loc[df['Original Language'] == 'Sp', 'Synopsis'] = es_synopsis_en


/opt/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [6]:
df.sample(10)

,Original Language,Title,Year,Synopsis,Review
3,En,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ..."
5,Fr,Le Dîner de Cons,1998,The film follows the story of a group of rich ...,"""I didn't like this movie at all. The concept ..."
2,Sp,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ..."
8,Fr,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ..."
0,Fr,La La Land,2016,This musical tells the story of a budding actr...,"""The Land is an absolutely beautiful film with..."
4,En,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun..."
2,Fr,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi..."
4,Sp,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha..."
7,Sp,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ..."
9,En,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ..."


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [7]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "sentiment-analysis"
sentiment_classifier = pipeline(model_name)

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    res = classifier(text)
    return "Positive" if res[0]['label'] == 'POSITIVE' else 'Negative'

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [8]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
df.insert(5, 'Sentiment', [analyze_sentiment(r, sentiment_classifier) for r in df['Review']])

In [9]:
df.sample(10)

,Original Language,Title,Year,Synopsis,Review,Sentiment
9,En,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",Negative
6,En,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",Negative
8,Fr,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",Negative
6,Fr,La Tour Montparnasse Infernale,2001,Two incompetent office workers find themselves...,"""I can't believe I've wasted time watching thi...",Negative
1,En,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",Positive
1,Fr,Intouchables,2011,This film tells the story of the unlikely frie...,"""Untouchables is an incredibly touching film w...",Positive
7,En,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",Negative
0,Sp,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic wo...,"""Rome is a beautiful and moving film that pays...",Positive
9,Sp,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",Negative
2,Sp,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",Positive


In [10]:
# export the results to a .csv file
df.to_csv("reviews_with_sentiment.csv", index=False)